### Setup

In [2]:
#Install packages
#pip install openai
#pip install scikit-learn
#pip install numpy
#pip install matplotlib
#pip install regez
#pip install fitz
#pip install pymupdf #Need this to use fitz
#pip install nltk
#pip install string
#pip install import PyPDF2
#pip install pytesseract

In [1]:
#Import packages
import os
import openai
#from openai import OpenAI
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
import string
import fitz
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_auc_score, roc_curve
import io
import PyPDF2
import pytesseract

### Test with One File

In [17]:
#Load and OCR sample document
#Open and OCR PDF files

pdf_file = open('Alabama_2.pdf', 'rb')

pdf_reader = PyPDF2.PdfReader(pdf_file)
print(len(pdf_reader.pages))

#Create empty string to store text
text = ''

#Extract each page and OCR it
for i in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[i]
    page_num = i+1

    #Extract text from page using PyPDF2
    extracted_text = page.extract_text()

    #Save text to empty string, along with page number
    #text += extracted_text
    text += 'Page_{}: \n'.format(page_num) + extracted_text + '\n\n'

    #Print extracted text
    #print(extracted_text)

216


In [5]:
#text

In [18]:
#Keep only text after page 108 by finding the phrase "Page_108"
page_index = text.find("Page_108:")
text = text[page_index + len("Page_108:"):]

In [20]:
#text

### Cleaned Text Basic Extraction

In [6]:
#Sample cleaned text example
txt_clean = ''' 2023  Evidence of Coverage for UnitedHealthcare Dual Complete® (HMO-POS D-SNP)
Chapter 4: Medical Benefits Chart (what is covered and what you pay) 102
Covered Routine Dental Benefits Included with Your Plan:     
Annual Maximum:  $ 3,000
In general, preventive and routine dental services are not covered under Original Medicare.  
Your UnitedHealthcare Routine Dental Benefit provides coverage for preventive and other 
necessary dental services such as: 
o Exams
o Cleanings (Prophylaxis, Periodontal Maintenance, & Deep Cleanings)
o Fillings
o Crowns
o Bridges
o Root Canals
o Partial Dentures
o Complete Dentures
o Implants 
Procedures used for cosmetic-only reasons (tooth bleaching/whitening, veneers, gingival 
recontouring), orthodontics, space maintenance, sales tax, charges for failure to keep 
appointments, dental case management, dental charges related to COVID screening, 
testing and vaccination, and unspecified procedures by report are not covered by the plan.  
After the annual maximum is exhausted, any remaining charges are your responsibility. 
Other limitations and exclusions are listed below.
This dental plan offers access to the robust UHC Dental National Medicare Advantage 
Network .  Network dentists have agreed to provide services at a negotiated rate.  If you see 
a network dentist, you cannot be billed more than that rate for covered services within the 
limitations of the plan.  Any fees associated with non-covered services are your 
responsibility.   
For assistance finding a provider, please use the dental provider search tool at 
myUHCMedicare.com .  You may also call 1-866-480-1086  for help with finding a provider or 
scheduling a dental appointment
This dental plan offers both in-network and out-of-network dental coverage, and all covered services have $0 copayment.  
Out-of-network dentists are not contracted to accept plan payment as payment in full, so they might charge you for more than what the plan pays, 
even for services listed as $0 copayment.  Seeing a provider from the robust dental network can therefore result in substantial savings.  
Benefits received out-of-network are subject to any in-network benefit maximums, limitations and/or exclusions.
Some covered services may consider prior tooth history and procedures in conjunction with frequency limitations. 
If you wish to discuss detailed information about your plan with your 
'''

In [ ]:
import requests

In [8]:
#Prompt creation for clean text

content = '''I am a Medicare patient reviewing my evidence of coverage document.  I want to review my dental benefits in the below, specifically 
the annual max and whether root canals, dental implants, crowns, and decorative gold teeth are covered.

The text is:

2023  Evidence of Coverage for UnitedHealthcare Dual Complete® (HMO-POS D-SNP)
Chapter 4: Medical Benefits Chart (what is covered and what you pay) 102
Covered Routine Dental Benefits Included with Your Plan:     
Annual Maximum:  $ 3,000
In general, preventive and routine dental services are not covered under Original Medicare.  
Your UnitedHealthcare Routine Dental Benefit provides coverage for preventive and other 
necessary dental services such as: 
o Exams
o Cleanings (Prophylaxis, Periodontal Maintenance, & Deep Cleanings)
o Fillings
o Crowns
o Bridges
o Root Canals
o Partial Dentures
o Complete Dentures
o Implants 
Procedures used for cosmetic-only reasons (tooth bleaching/whitening, veneers, gingival 
recontouring), orthodontics, space maintenance, sales tax, charges for failure to keep 
appointments, dental case management, dental charges related to COVID screening, 
testing and vaccination, and unspecified procedures by report are not covered by the plan.  
After the annual maximum is exhausted, any remaining charges are your responsibility. 
Other limitations and exclusions are listed below.
This dental plan offers access to the robust UHC Dental National Medicare Advantage 
Network .  Network dentists have agreed to provide services at a negotiated rate.  If you see 
a network dentist, you cannot be billed more than that rate for covered services within the 
limitations of the plan.  Any fees associated with non-covered services are your 
responsibility.   
For assistance finding a provider, please use the dental provider search tool at 
myUHCMedicare.com .  You may also call 1-866-480-1086  for help with finding a provider or 
scheduling a dental appointment
This dental plan offers both in-network and out-of-network dental coverage, and all covered services have $0 copayment.  
Out-of-network dentists are not contracted to accept plan payment as payment in full, so they might charge you for more than what the plan pays, 
even for services listed as $0 copayment.  Seeing a provider from the robust dental network can therefore result in substantial savings.  
Benefits received out-of-network are subject to any in-network benefit maximums, limitations and/or exclusions.
Some covered services may consider prior tooth history and procedures in conjunction with frequency limitations.

I need the resposnes in the below template:
Annual max: (respond in annual max in USD)
Root canals covered: (respond in yes/no)
Implants covered: (respond in yes/no)
Crowns covered: (respond in yes/no)
Gold teeth covered: (respond in yes/no)
'''

In [10]:
token = 'INSERT TOKEN HERE'

headers = {
    'accept': '*/*',
    'Authorization': f'Bearer {token}',
}

params = {
    'api-version': '2023-09-01-preview',
}

json_data = {
    'messages': [
        {
            'content': content,
            'role': 'user'
        },
    ],
    'use_case': 'Extraction_Test'
}

response = requests.post('INSERT YOUR API HERE', params=params, headers=headers, json=json_data)
#print(response.json())
print(response.json()['choices'][0]['message']['content'])

Annual max: $3,000
Root canals covered: Yes
Implants covered: Yes
Crowns covered: Yes
Gold teeth covered: No


### Summarizing Clean Text

In [11]:
#Prompt creation for summarizing clean text

content = '''I am a Medicare patient reviewing my evidence of coverage document.  I want to briefly summarize my dental benefits in the below, 
in terms of what is covered, exclusions, etc.

My evidence of coverage document is:

2023  Evidence of Coverage for UnitedHealthcare Dual Complete® (HMO-POS D-SNP)
Chapter 4: Medical Benefits Chart (what is covered and what you pay) 102
Covered Routine Dental Benefits Included with Your Plan:     
Annual Maximum:  $ 3,000
In general, preventive and routine dental services are not covered under Original Medicare.  
Your UnitedHealthcare Routine Dental Benefit provides coverage for preventive and other 
necessary dental services such as: 
o Exams
o Cleanings (Prophylaxis, Periodontal Maintenance, & Deep Cleanings)
o Fillings
o Crowns
o Bridges
o Root Canals
o Partial Dentures
o Complete Dentures
o Implants 
Procedures used for cosmetic-only reasons (tooth bleaching/whitening, veneers, gingival 
recontouring), orthodontics, space maintenance, sales tax, charges for failure to keep 
appointments, dental case management, dental charges related to COVID screening, 
testing and vaccination, and unspecified procedures by report are not covered by the plan.  
After the annual maximum is exhausted, any remaining charges are your responsibility. 
Other limitations and exclusions are listed below.
This dental plan offers access to the robust UHC Dental National Medicare Advantage 
Network .  Network dentists have agreed to provide services at a negotiated rate.  If you see 
a network dentist, you cannot be billed more than that rate for covered services within the 
limitations of the plan.  Any fees associated with non-covered services are your 
responsibility.   
For assistance finding a provider, please use the dental provider search tool at 
myUHCMedicare.com .  You may also call 1-866-480-1086  for help with finding a provider or 
scheduling a dental appointment
This dental plan offers both in-network and out-of-network dental coverage, and all covered services have $0 copayment.  
Out-of-network dentists are not contracted to accept plan payment as payment in full, so they might charge you for more than what the plan pays, 
even for services listed as $0 copayment.  Seeing a provider from the robust dental network can therefore result in substantial savings.  
Benefits received out-of-network are subject to any in-network benefit maximums, limitations and/or exclusions.
Some covered services may consider prior tooth history and procedures in conjunction with frequency limitations.

'''

In [12]:
token = 'INSERT TOKEN HERE'

headers = {
    'accept': '*/*',
    'Authorization': f'Bearer {token}',
}

params = {
    'api-version': '2023-09-01-preview',
}

json_data = {
    'messages': [
        {
            'content': content,
            'role': 'user'
        },
    ],
    'use_case': 'Extraction_Test'
}

response = requests.post('YOUR API HERE', params=params, headers=headers, json=json_data)
#print(response.json())
print(response.json()['choices'][0]['message']['content'])

In summary, the evidence of coverage document states that the UnitedHealthcare Dual Complete® (HMO-POS D-SNP) plan includes routine dental benefits. The plan covers preventive and necessary dental services such as exams, cleanings, fillings, crowns, bridges, root canals, partial dentures, complete dentures, and implants. However, cosmetic procedures, orthodontics, space maintenance, sales tax, charges for missed appointments, dental case management, dental charges related to COVID screening, testing, and vaccination, and unspecified procedures are not covered. There is an annual maximum coverage of $3,000, and any charges beyond that limit are the patient's responsibility. The plan offers access to the UHC Dental National Medicare Advantage Network, where network dentists provide services at negotiated rates. Out-of-network dentists may charge more than the plan pays, even for services with $0 copayment. It is recommended to use network dentists for cost savings. Prior tooth history an

### Extraction for Uncleaned Text

In [15]:
#Prompt creation for uncleaned text

content = '''I am a Medicare patient reviewing my evidence of coverage document.  I want to review my dental benefits in the below, specifically 
the annual max and whether root canals, dental implants, crowns, and decorative gold teeth are covered.

My evidence of coverage document is:

' \n2023  Evidence of Coverage for UnitedHealthcare Dual Complete® (HMO-POS D-SNP)
\nChapter 4: Medical Benefits Chart (what is covered and what you pay) 102
\nCovered Routine Dental Benefits Included with Your Plan:     
\nAnnual Maximum:  $ 3,000
\n· In general, preventive and routine dental services are not covered under Original Medicare.  
\nYour UnitedHealthcare Routine Dental Benefit provides coverage for preventive and other 
\nnecessary dental services such as: \no Exams\no Cleanings (Prophylaxis, Periodontal Maintenance, & Deep Cleanings)\no Fillings\no Crowns\no Bridges\no Root Canals\no Partial Dentures\no Complete Dentures\no Implants  \n· Procedures used for cosmetic-only reasons (tooth bleaching/whitening, veneers, gingival \nrecontouring), orthodontics, space maintenance, sales tax, charges for failure to keep \nappointments, dental case management, dental charges related to COVID screening, \ntesting and vaccination, and unspecified procedures by report are not covered by the plan.  \nAfter the annual maximum is exhausted, any remaining charges are your responsibility. \nOther limitations and exclusions are listed below.\xa0  \n· This dental plan offers access to the robust UHC Dental National Medicare Advantage \nNetwork .  Network dentists have agreed to provide services at a negotiated rate.  If you see \na network dentist, you cannot be billed more than that rate for covered services within the \nlimitations of the plan.  Any fees associated with non-covered services are your \nresponsibility.   \n· For assistance finding a provider, please use the dental provider search tool at \nmyUHCMedicare.com .  You may also call 1-866-480-1086  for help with finding a provider or \nscheduling a dental appointment\n· This dental plan offers both in-network and out-of-network dental coverage, and all covered \nservices have $0 copayment.  Out-of-network dentists are not contracted to accept plan \npayment as payment in full, so they might charge you for more than what the plan pays, \neven for services listed as $0 copayment.  Seeing a provider from the robust dental network \ncan therefore result in substantial savings.  Benefits received out-of-network are subject to \nany in-network benefit maximums, limitations and/or exclusions. \n· Some covered services may consider prior tooth history and procedures in conjunction with \nfrequency limitations. If you wish to discuss detailed information about your plan with your

I need the resposnes in the below template:
Annual max: (respond in annual max in USD)
Root canals covered: (respond in yes/no)
Implants covered: (respond in yes/no)
Crowns covered: (respond in yes/no)
Gold teeth covered: (respond in yes/no)

'''


In [ ]:
prompt = '''I am a Medicare analyst reviewing this evidence of coverage document.  
I want to review the dental benefits, specifically 
the annual max and whether root canals, dental implants, and crowns are covered.

My evidence of coverage document is: {text}

I need the resposnes in the below template:
Annual max: (respond in annual max in USD)
Root canals covered: (respond in yes/no)
Implants covered: (respond in yes/no)
Crowns covered: (respond in yes/no)

In [16]:
token = 'INSERT TOKEN HERE'
headers = {
    'accept': '*/*',
    'Authorization': f'Bearer {token}',
}

params = {
    'api-version': '2023-09-01-preview',
}

json_data = {
    'messages': [
        {
            'content': content,
            'role': 'user'
        },
    ],
    'use_case': 'Extraction_Test'
}

response = requests.post('YOUR API HERE', params=params, headers=headers, json=json_data)
#print(response.json())
print(response.json()['choices'][0]['message']['content'])

Annual max: $3,000
Root canals covered: Yes
Implants covered: Yes
Crowns covered: Yes
Gold teeth covered: No


### Comparing Two Plans Dental Benefits

In [28]:
#Prompt creation for summarizing clean text

content = '''I am a Medicare analyst reviewing evidence of coverage documents for two competing plans.  I want to briefly (a few sentences) compare the dental benefits
offered by the two plans.  For instance, does one plan cover more than the other?  Which plan is superior?

The first plan (United Health) is:

' \n2023  Evidence of Coverage for UnitedHealthcare Dual Complete® (HMO-POS D-SNP)
\nChapter 4: Medical Benefits Chart (what is covered and what you pay) 102
\nCovered Routine Dental Benefits Included with Your Plan:     
\nAnnual Maximum:  $ 3,000
\n· In general, preventive and routine dental services are not covered under Original Medicare.  
\nYour UnitedHealthcare Routine Dental Benefit provides coverage for preventive and other 
\nnecessary dental services such as: \no Exams\no Cleanings (Prophylaxis, Periodontal Maintenance, & Deep Cleanings)\no Fillings\no Crowns\no Bridges\no Root Canals\no Partial Dentures\no Complete Dentures\no Implants  \n· Procedures used for cosmetic-only reasons (tooth bleaching/whitening, veneers, gingival \nrecontouring), orthodontics, space maintenance, sales tax, charges for failure to keep \nappointments, dental case management, dental charges related to COVID screening, \ntesting and vaccination, and unspecified procedures by report are not covered by the plan.  \nAfter the annual maximum is exhausted, any remaining charges are your responsibility. \nOther limitations and exclusions are listed below.\xa0  \n· This dental plan offers access to the robust UHC Dental National Medicare Advantage \nNetwork .  Network dentists have agreed to provide services at a negotiated rate.  If you see \na network dentist, you cannot be billed more than that rate for covered services within the \nlimitations of the plan.  Any fees associated with non-covered services are your \nresponsibility.   \n· For assistance finding a provider, please use the dental provider search tool at \nmyUHCMedicare.com .  You may also call 1-866-480-1086  for help with finding a provider or \nscheduling a dental appointment\n· This dental plan offers both in-network and out-of-network dental coverage, and all covered \nservices have $0 copayment.  Out-of-network dentists are not contracted to accept plan \npayment as payment in full, so they might charge you for more than what the plan pays, \neven for services listed as $0 copayment.  Seeing a provider from the robust dental network \ncan therefore result in substantial savings.  Benefits received out-of-network are subject to \nany in-network benefit maximums, limitations and/or exclusions. \n· Some covered services may consider prior tooth history and procedures in conjunction with \nfrequency limitations. If you wish to discuss detailed information about your plan with your

The second plan (Humana) is:

' \n2023 Evidence of Coverage for HumanaChoice H5216-214 (PPO) 51
\nChapter 4. Medical Benefits Chart (what is covered and what you pay)\nServices that are covered for you What you must pay when you get 
\nthese services\n•We cover only manual manipulation of the spine to correct subluxation
\n•Other services performed by a chiropractor are not covered \nPrior authorization requirements may apply.$20 copayment\n–Specialist\'s Office\nOut-of-Network:\nMedicare Covered Chiropractic \nServices\n40% coinsurance\n–Specialist\'s Office\n Colorectal cancer screening\nFor people 50 and older, the following are covered:\n•Flexible sigmoidoscopy (or screening barium enema as an alternative) \nevery 48 months\nOne of the following every 12 months:\n•Guaiac-based fecal occult blood test (gFOBT)\n•Fecal immunochemical test (FIT)\nDNA based colorectal screening every 3 years\nFor people at high risk of colorectal cancer, we cover:\n•Screening colonoscopy (or screening barium enema as an alternative) \nevery 24 months\nFor people not at high risk of colorectal cancer, we cover:\n•Screening colonoscopy every 10 years (120 months), but not within 48 \nmonths of a screening sigmoidoscopyIn-Network:\nThere is no coinsurance, \ncopayment, or deductible for a \nMedicare-covered colorectal cancer \nscreening exam.\nOut-of-Network:\n40% coinsurance\n–Specialist\'s Office\n–Ambulatory Surgical Center\n–Outpatient Hospital\nDental services\nIn general, preventive dental services (such as cleaning, routine dental \nexams, and dental x-rays) are not covered by Original Medicare. We cover:\n•Medically necessary dental services, as covered by Original Medicare\nPrior authorization requirements may apply.In-Network:\nMedicare Covered Dental Services\n$35 copayment\n–Specialist\'s Office\nOut-of-Network:\nMedicare Covered Dental Services\n40% coinsurance\n–Specialist\'s Office\nSupplemental dental benefits\n*You are covered for supplemental \ndental benefits. See the \nsupplemental dental benefit \ndescription at the end of this chart \nfor details.\n Depression screening\nWe cover one screening for depression per year. The screening must be \ndone in a primary care setting that can provide follow-up treatment and/or \nreferrals.In-Network:\nThere is no coinsurance, \ncopayment, or deductible for an \nannual depression screening visit

'''

In [29]:
token = 'INSERT TOKEN HERE'

headers = {
    'accept': '*/*',
    'Authorization': f'Bearer {token}',
}

params = {
    'api-version': '2023-09-01-preview',
}

json_data = {
    'messages': [
        {
            'content': content,
            'role': 'user'
        },
    ],
    'use_case': 'Extraction_Test'
}

response = requests.post('YOUR API HERE', params=params, headers=headers, json=json_data)
#print(response.json())
print(response.json()['choices'][0]['message']['content'])

The UnitedHealth plan offers coverage for a wide range of dental services, including exams, cleanings, fillings, crowns, bridges, root canals, dentures, and implants. It also has an annual maximum of $3,000 and offers access to the UHC Dental National Medicare Advantage Network. The Humana plan covers medically necessary dental services, with a $35 copayment for in-network services and a 40% coinsurance for out-of-network services. It also offers supplemental dental benefits, but the details are not provided in this document. Based on the information provided, the UnitedHealth plan appears to offer more comprehensive dental coverage and may be considered superior. However, a full comparison would require additional information on the supplemental dental benefits offered by the Humana plan.
